In [354]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/Users/muhammadqasimbhatti/Documents/Kaggle/Mental Health Competition/train.csv')
test = pd.read_csv('/Users/muhammadqasimbhatti/Documents/Kaggle/Mental Health Competition/test.csv')
df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


### The Classes are imbalanced


In [355]:
num_columns = df.select_dtypes(include=np.number).columns
cat_columns = df.select_dtypes(include=object).columns
print(num_columns)
print(cat_columns)


Index(['id', 'Age', 'Academic Pressure', 'Work Pressure', 'CGPA',
       'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours',
       'Financial Stress', 'Depression'],
      dtype='object')
Index(['Name', 'Gender', 'City', 'Working Professional or Student',
       'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree',
       'Have you ever had suicidal thoughts ?',
       'Family History of Mental Illness'],
      dtype='object')


In [356]:
df['Depression'].value_counts()

Depression
0    115133
1     25567
Name: count, dtype: int64

In [357]:
temp = df[df['Profession'].isnull()][['Working Professional or Student', 'Profession']]
temp.head(10)
temp['Working Professional or Student'].value_counts()


Working Professional or Student
Student                 27867
Working Professional     8763
Name: count, dtype: int64

We have 27867 Missing Student Profession and 8763 Missing Working Professional Profession

In [358]:
df['Profession'] = df['Profession'].fillna(df['Working Professional or Student'])
test['Profession'] = test['Profession'].fillna(test['Working Professional or Student'])
df.head()


,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,Student,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [359]:
# Calculate median from training data only to avoid data leakage
median_academic_pressure_student = df[df['Profession'] == 'Student']['Academic Pressure'].median()

# Fill missing values in both train and test using training median
df.loc[df['Profession'] == 'Student', 'Academic Pressure'] = df.loc[df['Profession'] == 'Student', 'Academic Pressure'].fillna(median_academic_pressure_student)
test.loc[test['Profession'] == 'Student', 'Academic Pressure'] = test.loc[test['Profession'] == 'Student', 'Academic Pressure'].fillna(median_academic_pressure_student)

print("\nNull values in Academic Pressure after filling:")
print("Training set:", df['Academic Pressure'].isnull().sum())
print("Test set:", test['Academic Pressure'].isnull().sum())



Null values in Academic Pressure after filling:
Training set: 112790
Test set: 75020


In [360]:
# # Calculate median from training data only to avoid data leakage

# for profession in df['Profession'].unique():
#     if profession != 'Student':
#         mode_work_pressure = df[df['Profession'] == profession]['Work Pressure'].mode()[0]
#         df.loc[df['Profession'] == profession, 'Work Pressure'] = df.loc[df['Profession'] == profession, 'Work Pressure'].fillna(mode_work_pressure)
#         test.loc[test['Profession'] == profession, 'Work Pressure'] = test.loc[test['Profession'] == profession, 'Work Pressure'].fillna(mode_work_pressure)

median_work_pressure_professional = df[df['Profession'] != 'Student']['Work Pressure'].median()

# # Fill missing values in both train and test using training median
df.loc[df['Profession'] != 'Student', 'Work Pressure'] = df.loc[df['Profession'] != 'Student', 'Work Pressure'].fillna(median_work_pressure_professional)
test.loc[test['Profession'] != 'Student', 'Work Pressure'] = test.loc[test['Profession'] != 'Student', 'Work Pressure'].fillna(median_work_pressure_professional)

print("\nNull values in Work Pressure after filling:")
print("Training set:", df['Work Pressure'].isnull().sum())
print("Test set:", test['Work Pressure'].isnull().sum())



Null values in Work Pressure after filling:
Training set: 27867
Test set: 18744


### CGPA


In [361]:
# Calculate median from training data only to avoid data leakage
median_cgpa_student = df[df['Profession'] == 'Student']['CGPA'].median()

# Fill missing values in both train and test using training median
df.loc[df['Profession'] == 'Student', 'CGPA'] = df.loc[df['Profession'] == 'Student', 'CGPA'].fillna(median_cgpa_student)
test.loc[test['Profession'] == 'Student', 'CGPA'] = test.loc[test['Profession'] == 'Student', 'CGPA'].fillna(median_cgpa_student)

print("\nNull values in CGPA after filling:")
print("Training set:", df['CGPA'].isnull().sum())
print("Test set:", test['CGPA'].isnull().sum())



Null values in CGPA after filling:
Training set: 112789
Test set: 75019


In [362]:
# df['CGPA'] = df['CGPA'].fillna(0)
# test['CGPA'] = test['CGPA'].fillna(0)


### Study Satisfaction

In [363]:
# Calculate median from training data only to avoid data leakage
median_study_satisfaction_student = df[df['Profession'] == 'Student']['Study Satisfaction'].median()


# Fill missing values in both train and test using training median
df.loc[df['Profession'] == 'Student', 'Study Satisfaction'] = df.loc[df['Profession'] == 'Student', 'Study Satisfaction'].fillna(median_study_satisfaction_student)
test.loc[test['Profession'] == 'Student', 'Study Satisfaction'] = test.loc[test['Profession'] == 'Student', 'Study Satisfaction'].fillna(median_study_satisfaction_student)

print("\nNull values in Study Satisfaction after filling:")
print("Training set:", df['Study Satisfaction'].isnull().sum())
print("Test set:", test['Study Satisfaction'].isnull().sum())



Null values in Study Satisfaction after filling:
Training set: 112789
Test set: 75019


In [364]:
# Calculate median from training data only to avoid data leakage

# for profession in df['Profession'].unique():
#     if profession != 'Student':
#         mode_job_satisfaction = df[df['Profession'] == profession]['Job Satisfaction'].mode()[0]
#         df.loc[df['Profession'] == profession, 'Job Satisfaction'] = df.loc[df['Profession'] == profession, 'Job Satisfaction'].fillna(mode_job_satisfaction)
#         test.loc[test['Profession'] == profession, 'Job Satisfaction'] = test.loc[test['Profession'] == profession, 'Job Satisfaction'].fillna(mode_job_satisfaction)

median_job_satisfaction_working = df[df['Profession'] != 'Student']['Job Satisfaction'].median() 

# # Fill missing values in b    oth train and test using training median
df.loc[df['Profession'] != 'Student', 'Job Satisfaction'] = df.loc[df['Profession'] != 'Student', 'Job Satisfaction'].fillna(median_job_satisfaction_working)
test.loc[test['Profession'] != 'Student', 'Job Satisfaction'] = test.loc[test['Profession'] != 'Student', 'Job Satisfaction'].fillna(median_job_satisfaction_working)

print("\nNull values in Job Satisfaction after filling:")
print("Training set:", df['Job Satisfaction'].isnull().sum())
print("Test set:", test['Job Satisfaction'].isnull().sum())



Null values in Job Satisfaction after filling:
Training set: 27862
Test set: 18741


### Total Pressure

In [365]:
# df['Total Pressure'] = df['Academic Pressure'].fillna(0) + df['Work Pressure'].fillna(0)
# df.drop(columns=['Academic Pressure', 'Work Pressure'], inplace=True)

# test['Total Pressure'] = test['Academic Pressure'].fillna(0) + test['Work Pressure'].fillna(0)
# test.drop(columns=['Academic Pressure', 'Work Pressure'], inplace=True)
# df.head(10)



### Job/Study Satisfaction

In [366]:
# df['study/job Pressure'] = df['Study Satisfaction'].fillna(0) + df['Job Satisfaction'].fillna(0)
# df.drop(columns=['Study Satisfaction', 'Job Satisfaction'], inplace=True)

# test['study/job Pressure'] = test['Study Satisfaction'].fillna(0) + test['Job Satisfaction'].fillna(0)
# test.drop(columns=['Study Satisfaction', 'Job Satisfaction'], inplace=True)
# df.head(10)



In [367]:
df.isnull().sum()

id                                            0
Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                    0
Academic Pressure                        112790
Work Pressure                             27867
CGPA                                     112789
Study Satisfaction                       112789
Job Satisfaction                          27862
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64

### Family History of Mental Illness and Have you ever had suicidal thoughts ?

In [368]:
df['Family History of Mental Illness'] = df['Family History of Mental Illness'].map({'Yes': 1, 'No': 0 })
df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].map({'Yes': 1, 'No': 0})

test['Family History of Mental Illness'] = test['Family History of Mental Illness'].map({'Yes': 1, 'No': 0})
test['Have you ever had suicidal thoughts ?'] = test['Have you ever had suicidal thoughts ?'].map({'Yes': 1, 'No': 0})


In [369]:
# Fill Dietary Habits with mode from training data
mode_dietary_habits = df['Dietary Habits'].mode()[0]  # Calculate mode from training data
df['Dietary Habits'] = df['Dietary Habits'].fillna(mode_dietary_habits)
print("\nNull values in Dietary Habits (training) after filling:")
print(df['Dietary Habits'].isnull().sum())

# Use the training mode for test set
test['Dietary Habits'] = test['Dietary Habits'].fillna(mode_dietary_habits)
print("\nNull values in Dietary Habits (test) after filling:")
print(test['Dietary Habits'].isnull().sum())

# Fill Degree with mode from training data
mode_degree = df['Degree'].mode()[0]  # Calculate mode from training data
df['Degree'] = df['Degree'].fillna(mode_degree)
print("\nNull values in Degree (training) after filling:")
print(df['Degree'].isnull().sum())

# Use the training mode for test set
test['Degree'] = test['Degree'].fillna(mode_degree)
print("\nNull values in Degree (test) after filling:")
print(test['Degree'].isnull().sum())

# Fill Financial Stress with median from training data
median_financial_stress = df['Financial Stress'].median()  # Calculate median from training data
df['Financial Stress'] = df['Financial Stress'].fillna(median_financial_stress)
print("\nNull values in Financial Stress (training) after filling:")
print(df['Financial Stress'].isnull().sum())

# Use the training median for test set
test['Financial Stress'] = test['Financial Stress'].fillna(median_financial_stress)
print("\nNull values in Financial Stress (test) after filling:")
print(test['Financial Stress'].isnull().sum())



Null values in Dietary Habits (training) after filling:
0

Null values in Dietary Habits (test) after filling:
0

Null values in Degree (training) after filling:
0

Null values in Degree (test) after filling:
0

Null values in Financial Stress (training) after filling:
0

Null values in Financial Stress (test) after filling:
0


In [370]:
df.isnull().sum()

id                                            0
Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                    0
Academic Pressure                        112790
Work Pressure                             27867
CGPA                                     112789
Study Satisfaction                       112789
Job Satisfaction                          27862
Sleep Duration                                0
Dietary Habits                                0
Degree                                        0
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              0
Family History of Mental Illness              0
Depression                                    0
dtype: int64

### Sleep Duration

In [371]:
df['Sleep Duration'].value_counts()

Sleep Duration
Less than 5 hours    38784
7-8 hours            36969
More than 8 hours    32726
5-6 hours            32142
3-4 hours               12
6-7 hours                8
4-5 hours                7
2-3 hours                5
4-6 hours                5
6-8 hours                4
1-6 hours                4
No                       4
9-11 hours               2
10-11 hours              2
Sleep_Duration           2
Unhealthy                2
45                       2
8-9 hours                2
10-6 hours               1
9-5                      1
45-48 hours              1
3-6 hours                1
Work_Study_Hours         1
49 hours                 1
than 5 hours             1
Pune                     1
9-6 hours                1
8 hours                  1
35-36 hours              1
Indore                   1
1-3 hours                1
55-66 hours              1
Moderate                 1
40-45 hours              1
1-2 hours                1
9-5 hours                1
Name: count, 

In [372]:
# # Create a mapping for valid sleep duration categories

# valid_sleep_duration_categories = ["Less than 5 hours", "5-6 hours", "7-8 hours", "More than 8 hours"]

# # Check and fill invalid sleep duration categories with mode
# mode_sleep_duration = df['Sleep Duration'].mode()[0]
# df['Sleep Duration'] = df['Sleep Duration'].apply(lambda x: x if x in valid_sleep_duration_categories else mode_sleep_duration)
# test['Sleep Duration'] = test['Sleep Duration'].apply(lambda x: x if x in valid_sleep_duration_categories else mode_sleep_duration)





In [373]:

# Convert sleep hours into numeric values with extended categories
sleep_duration_mapping = {
    "Less than 5 hours": 4.5,
    "5-6 hours": 5.5,
    "7-8 hours": 7.5,
    "More than 8 hours": 9,
    "3-4 hours": 3.5,
    "6-7 hours": 6.5,
    "4-5 hours": 4.5,
    "2-3 hours": 2.5,
    "4-6 hours": 5,
    "6-8 hours": 7,
    "1-6 hours": 3.5,  # Assuming average
    "9-11 hours": 10,
    "10-11 hours": 10.5,
    "8-9 hours": 8.5,
    "10-6 hours": 8,  # Assuming average
    "9-5": 7,  # Assuming average
    "3-6 hours": 4.5,
    "1-2 hours": 1.5,
    "9-5 hours": 7,  # Assuming average
}

# Calculate mode for sleep duration
mode_sleep_duration = df['Sleep Duration'].mode()[0]

# Map sleep duration to numeric values, using mode for unmapped categories
df['Sleep Duration'] = df['Sleep Duration'].apply(lambda x: sleep_duration_mapping.get(x, 4.5))
test['Sleep Duration'] = test['Sleep Duration'].apply(lambda x: sleep_duration_mapping.get(x, 4.5))



In [374]:
df['Sleep Duration'].value_counts()

Sleep Duration
4.5     38815
7.5     36969
9.0     32726
5.5     32142
3.5        16
6.5         8
7.0         6
5.0         5
2.5         5
10.5        2
8.5         2
10.0        2
1.5         1
8.0         1
Name: count, dtype: int64

In [375]:
df.head(10)



,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,9.0,Healthy,BHM,0,1.0,2.0,0,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,4.5,Unhealthy,LLB,1,7.0,3.0,0,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,Student,5.0,NaN,8.97,2.0,NaN,5.5,Healthy,B.Pharm,1,3.0,1.0,0,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,4.5,Moderate,BBA,1,10.0,1.0,1,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5.5,Unhealthy,BBA,1,9.0,4.0,1,0
5,5,Vani,Female,59.0,Ahmedabad,Working Professional,Finanancial Analyst,NaN,2.0,NaN,NaN,5.0,5.5,Healthy,MCA,0,7.0,5.0,0,0
6,6,Ritvik,Male,47.0,Thane,Working Professional,Chemist,NaN,5.0,NaN,NaN,2.0,7.5,Moderate,MD,0,6.0,2.0,0,0
7,7,Rajveer,Male,38.0,Nashik,Working Professional,Teacher,NaN,3.0,NaN,NaN,4.0,7.5,Unhealthy,B.Pharm,0,10.0,3.0,1,0
8,8,Aishwarya,Female,24.0,Bangalore,Student,Student,2.0,NaN,5.90,5.0,NaN,5.5,Moderate,BSc,0,3.0,2.0,1,0
9,9,Simran,Female,42.0,Patna,Working Professional,Electrician,NaN,4.0,NaN,NaN,1.0,5.5,Healthy,ME,1,7.0,2.0,1,0


### Dietary Habits

In [376]:
# Create a mapping for valid dietary habit categories
valid_categories = ["Healthy", "Moderate", "Unhealthy"]
mode_dietary_habits = df['Dietary Habits'].mode()[0]
df["Dietary Habits"] = df["Dietary Habits"].apply(lambda x: x if x in valid_categories else mode_dietary_habits)
test["Dietary Habits"] = test["Dietary Habits"].apply(lambda x: x if x in valid_categories else mode_dietary_habits)

print("\nDietary Habits categories after cleaning:")
print(df['Dietary Habits'].value_counts())



Dietary Habits categories after cleaning:
Dietary Habits
Moderate     49732
Unhealthy    46227
Healthy      44741
Name: count, dtype: int64


In [377]:
diet_score = df['Dietary Habits'].map({'Healthy': 1, 'Moderate': 0, 'Unhealthy': -1})
test_diet_score = test['Dietary Habits'].map({'Healthy': 1, 'Moderate': 0, 'Unhealthy': -1})
df['Dietary Habits'] = diet_score
test['Dietary Habits'] = test_diet_score

In [378]:
df.head(10)

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,9.0,1,BHM,0,1.0,2.0,0,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,4.5,-1,LLB,1,7.0,3.0,0,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,Student,5.0,NaN,8.97,2.0,NaN,5.5,1,B.Pharm,1,3.0,1.0,0,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,4.5,0,BBA,1,10.0,1.0,1,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5.5,-1,BBA,1,9.0,4.0,1,0
5,5,Vani,Female,59.0,Ahmedabad,Working Professional,Finanancial Analyst,NaN,2.0,NaN,NaN,5.0,5.5,1,MCA,0,7.0,5.0,0,0
6,6,Ritvik,Male,47.0,Thane,Working Professional,Chemist,NaN,5.0,NaN,NaN,2.0,7.5,0,MD,0,6.0,2.0,0,0
7,7,Rajveer,Male,38.0,Nashik,Working Professional,Teacher,NaN,3.0,NaN,NaN,4.0,7.5,-1,B.Pharm,0,10.0,3.0,1,0
8,8,Aishwarya,Female,24.0,Bangalore,Student,Student,2.0,NaN,5.90,5.0,NaN,5.5,0,BSc,0,3.0,2.0,1,0
9,9,Simran,Female,42.0,Patna,Working Professional,Electrician,NaN,4.0,NaN,NaN,1.0,5.5,1,ME,1,7.0,2.0,1,0


### Horaaah!! Finally there is no missing values in the dataset

In [379]:
df_copy = df.copy()
train = df_copy.drop(columns=['Name', 'id', 'Degree'], axis=1)
test_ids = test['id'].copy() 
test = test.drop(columns=['Name', 'id', 'Degree'], axis=1)


# scaling numerical features


In [380]:
X = train.drop(columns=['Depression'])
y = train['Depression']

from sklearn.preprocessing import StandardScaler

# Identify numeric columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

print("Numeric Features:")
print(numeric_features.tolist())

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the whole training data and transform the training data
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Transform the test data
test[numeric_features] = scaler.transform(test[numeric_features])

print("Numeric features have been scaled using StandardScaler.")

X.head(10)


Numeric Features:
['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration', 'Dietary Habits', 'Have you ever had suicidal thoughts ?', 'Work/Study Hours', 'Financial Stress', 'Family History of Mental Illness']
Numeric features have been scaled using StandardScaler.


,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,Female,0.695360,Ludhiana,Working Professional,Chef,NaN,1.423747,NaN,NaN,-0.688258,1.395813,1.256906,-0.988861,-1.363057,-0.699617,-0.994217
1,Male,-1.161867,Varanasi,Working Professional,Teacher,NaN,0.712230,NaN,NaN,0.018072,-1.182285,-1.230634,1.011265,0.193928,0.007793,-0.994217
2,Male,-0.596624,Visakhapatnam,Student,Student,1.346116,NaN,0.895643,-0.694915,NaN,-0.609374,1.256906,1.011265,-0.844062,-1.407027,-0.994217
3,Male,-1.484863,Mumbai,Working Professional,Teacher,NaN,1.423747,NaN,NaN,-1.394587,-1.182285,0.013136,1.011265,0.972421,-1.407027,1.005816
4,Female,-0.838871,Kanpur,Working Professional,Business Analyst,NaN,-1.422322,NaN,NaN,-1.394587,-0.609374,-1.230634,1.011265,0.712923,0.715203,1.005816
5,Female,1.502850,Ahmedabad,Working Professional,Finanancial Analyst,NaN,-0.710805,NaN,NaN,1.430730,-0.609374,1.256906,-0.988861,0.193928,1.422613,-0.994217
6,Male,0.533862,Thane,Working Professional,Chemist,NaN,1.423747,NaN,NaN,-0.688258,0.536447,0.013136,-0.988861,-0.065570,-0.699617,-0.994217
7,Male,-0.192879,Nashik,Working Professional,Teacher,NaN,0.000713,NaN,NaN,0.724401,0.536447,-1.230634,-0.988861,0.972421,0.007793,1.005816
8,Female,-1.323365,Bangalore,Student,Student,-0.827618,NaN,-1.201207,1.511239,NaN,-0.609374,0.013136,-0.988861,-0.844062,-0.699617,1.005816
9,Female,0.130117,Patna,Working Professional,Electrician,NaN,0.712230,NaN,NaN,-1.394587,-0.609374,1.256906,1.011265,0.193928,-0.699617,1.005816


# Encoding categorical features

In [381]:
# One-hot encoding for 'Profession' and 'Degree'
X = pd.get_dummies(X, columns=['Profession', 'City',  'Working Professional or Student', 'Gender'], drop_first=True)
test = pd.get_dummies(test, columns=['Profession', 'City',  'Working Professional or Student', 'Gender'], drop_first=True)
test = test.reindex(columns=X.columns, fill_value=0)

print("Encoded features:")
test.head(10)


Encoded features:


,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Have you ever had suicidal thoughts ?,Work/Study Hours,...,City_Vaanya,City_Vadodara,City_Vaishnavi,City_Varanasi,City_Vasai-Virar,City_Vidhi,City_Vidya,City_Visakhapatnam,Working Professional or Student_Working Professional,Gender_Male
0,1.018356,NaN,-0.710805,NaN,NaN,1.430730,-1.182285,0.013136,-0.988861,0.712923,...,0,False,False,False,False,False,False,True,True,True
1,1.422101,NaN,-0.710805,NaN,NaN,0.724401,-1.182285,0.013136,-0.988861,-0.065570,...,0,False,False,False,False,False,False,False,True,False
2,1.018356,NaN,0.712230,NaN,NaN,-1.394587,0.536447,0.013136,1.011265,1.491416,...,0,False,False,False,False,False,False,False,True,True
3,-1.404114,1.346116,NaN,-0.559175,-1.43030,NaN,1.395813,0.013136,1.011265,0.972421,...,0,False,False,False,False,False,False,False,False,False
4,0.533862,NaN,1.423747,NaN,NaN,1.430730,0.536447,0.013136,1.011265,-0.844062,...,0,False,False,False,False,False,False,False,True,True
5,-0.919620,NaN,-0.710805,NaN,NaN,0.018072,1.395813,0.013136,-0.988861,-0.844062,...,0,False,False,False,False,False,False,False,True,True
6,0.533862,NaN,-1.422322,NaN,NaN,-1.394587,-1.182285,1.256906,-0.988861,0.972421,...,0,False,False,False,False,False,False,True,True,True
7,-1.000369,NaN,1.423747,NaN,NaN,0.018072,0.536447,1.256906,1.011265,-1.622555,...,0,False,False,False,False,False,False,False,True,False
8,-1.565612,-1.552196,NaN,-0.183517,0.04047,NaN,-1.182285,1.256906,-0.988861,0.453425,...,0,False,False,False,False,False,False,False,False,False
9,-1.565612,NaN,1.423747,NaN,NaN,-1.394587,-1.182285,1.256906,1.011265,0.972421,...,0,False,False,False,False,False,False,False,True,True


In [382]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import optuna

# Handle missing values by using XGBoost's built-in handling
X.fillna(np.nan, inplace=True)  # Replace NaN with a value that won't affect the model

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 30, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 20),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 20),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 20),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 2),
        'random_state': 42,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'verbosity': 0  # Suppress output for Optuna trials
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_accuracies = []
    for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train_fold, X_test_fold = X.iloc[train_index].values, X.iloc[test_index].values
        y_train_fold, y_test_fold = y.iloc[train_index].values, y.iloc[test_index].values
        model = XGBClassifier(**param)
        model.fit(X_train_fold, y_train_fold)
        preds = model.predict(X_test_fold)
        accuracy = accuracy_score(y_test_fold, preds)
        fold_accuracies.append(accuracy)

    return np.mean(fold_accuracies)

# Run Optuna to find the best hyperparameters
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5, n_jobs=1)

# Retrieve the best trial's parameters from the study
best_params = study.best_trial.params

# Train and evaluate the best model with StratifiedKFold
fold_accuracies = []
best_model = XGBClassifier(**best_params, random_state=42, verbosity=0)  # Suppress output
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train_fold, X_test_fold = X.iloc[train_index].values, X.iloc[test_index].values
    y_train_fold, y_test_fold = y.iloc[train_index].values, y.iloc[test_index].values
    best_model.fit(X_train_fold, y_train_fold)
    preds = best_model.predict(X_test_fold)
    accuracy = accuracy_score(y_test_fold, preds)
    fold_accuracies.append(accuracy)
    print(f"Using Best Model: Validation Fold {fold + 1} Accuracy: {accuracy:.4f}")

# Calculate and print the mean accuracy
mean_accuracy = np.mean(fold_accuracies)
print(f"Mean accuracy for the best model: {mean_accuracy:.4f}")

# Save best parameters and accuracy to a file
with open("xgboost_model_performance.txt", "a") as file:
    file.write(f"Params: {best_params}, Validation Accuracy: {mean_accuracy:.4f}\n")


[I 2024-11-15 00:53:00,932] A new study created in memory with name: no-name-c74af564-f894-4e21-b0fc-4b9899fe11d7
[I 2024-11-15 00:53:17,645] Trial 0 finished with value: 0.934911158493248 and parameters: {'n_estimators': 971, 'max_depth': 7, 'learning_rate': 0.25877213973167723, 'subsample': 0.8921386862024125, 'colsample_bytree': 0.5645261454993931, 'gamma': 4.936545729144135, 'reg_alpha': 3.662858284271364, 'reg_lambda': 7.9632341307045085, 'scale_pos_weight': 0.5293631252630971}. Best is trial 0 with value: 0.934911158493248.
[I 2024-11-15 00:53:30,802] Trial 1 finished with value: 0.938585643212509 and parameters: {'n_estimators': 642, 'max_depth': 9, 'learning_rate': 0.30588362063037605, 'subsample': 0.7775113750775522, 'colsample_bytree': 0.7901799572038946, 'gamma': 9.85258249528264, 'reg_alpha': 8.973353382927744, 'reg_lambda': 6.190663196107251, 'scale_pos_weight': 1.0625077993665677}. Best is trial 1 with value: 0.938585643212509.
[I 2024-11-15 00:53:42,678] Trial 2 finished

Using Best Model: Validation Fold 1 Accuracy: 0.9373
Using Best Model: Validation Fold 2 Accuracy: 0.9370
Using Best Model: Validation Fold 3 Accuracy: 0.9395
Using Best Model: Validation Fold 4 Accuracy: 0.9414
Using Best Model: Validation Fold 5 Accuracy: 0.9377
Mean accuracy for the best model: 0.9386


In [385]:
# Sort trials based on their performance and retrieve the top five
best_trials = sorted(study.trials, key=lambda trial: trial.value, reverse=True)[:5]

# Iterate over the top five trials to initialize models and make predictions
for i, best_trial in enumerate(best_trials):
    # Initialize the model with the best parameters from the trial
    model = XGBClassifier(**best_trial.params, random_state=42, verbosity=0)  # Suppress output

    # Fit the model on the complete training dataset
    model.fit(X, y)

    # Make predictions on the test dataset
    final_test_predictions = model.predict(test)

    # Create dataframe with test IDs and predictions
    results_df = pd.DataFrame({
        'id': test_ids,
        'Depression': final_test_predictions
    })

    # Save results to CSV without index
    filename = f'submission_xgb_accuracy_{best_trial.value:.4f}.csv'
    results_df.to_csv(filename, index=False)

    print(f"Results saved to {filename}")
    print("\nFirst few rows of results:")
    print(results_df.head())

    # Save best parameters and accuracy to a file for XGB model
    with open("xgb_model_performance.txt", "a") as file:
        file.write(f"Trial {i+1} Params: {best_trial.params}, Validation Accuracy: {best_trial.value:.4f}\n")


Results saved to submission_xgb_accuracy_0.9386.csv

First few rows of results:
       id  Depression
0  140700           0
1  140701           0
2  140702           0
3  140703           1
4  140704           0
Results saved to submission_xgb_accuracy_0.9382.csv

First few rows of results:
       id  Depression
0  140700           0
1  140701           0
2  140702           0
3  140703           1
4  140704           0
Results saved to submission_xgb_accuracy_0.9381.csv

First few rows of results:
       id  Depression
0  140700           0
1  140701           0
2  140702           0
3  140703           1
4  140704           0
Results saved to submission_xgb_accuracy_0.9368.csv

First few rows of results:
       id  Depression
0  140700           0
1  140701           0
2  140702           0
3  140703           1
4  140704           0
Results saved to submission_xgb_accuracy_0.9349.csv

First few rows of results:
       id  Depression
0  140700           0
1  140701           0
2  140

In [111]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd

# Define categorical features
categorical_features = ['Gender', 'City', 'Working Professional or Student', 'Profession', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']

# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')
    test[feature] = test[feature].astype('category')
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 30, 600),
        'depth': trial.suggest_int('depth', 3, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 20.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 300),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.000001, 20.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 20.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 2),
        'cat_features': categorical_features,
        'thread_count': 10,
        'random_seed': 42,
        'verbose': False,
        'early_stopping_rounds': 50,
        'loss_function': 'Logloss',
        'border_count': trial.suggest_int('border_count', 128, 254),
        'eval_metric': 'Accuracy'
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_accuracies = []
    for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        model = CatBoostClassifier(**param)
        model.fit(X_train_fold, y_train_fold, cat_features=categorical_features)
        preds = model.predict(X_test_fold)
        accuracy = accuracy_score(y_test_fold, preds)
        fold_accuracies.append(accuracy)
    return np.mean(fold_accuracies)

# Run only one trial at a time to fully utilize all cores
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1000, n_jobs=1)

# Retrieve the best model from the study
top_model = study.best_trial

# Train the best model and test it on validation data
fold_accuracies = []
model_params = top_model.params
best_model = CatBoostClassifier(**model_params, random_seed=42, verbose=False, early_stopping_rounds=50, eval_metric='Accuracy')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    best_model.fit(X_train_fold, y_train_fold, cat_features=categorical_features)
    preds = best_model.predict(X_test_fold)
    accuracy = accuracy_score(y_test_fold, preds)
    fold_accuracies.append(accuracy)
    print(f"Using Best Model: Validation Fold {fold+1} Accuracy: {accuracy:.4f}")

mean_accuracy = np.mean(fold_accuracies)
print(f"Mean accuracy for the best model: {mean_accuracy:.4f}")

# Save parameters and accuracy to a file
with open("catboost_model_performance.txt", "a") as file:
    file.write(f"Params: {top_model.params}, Validation Accuracy: {mean_accuracy:.4f}\n")


In [112]:
# Sort trials based on their performance and retrieve the top one
best_trial = max(study.trials, key=lambda trial: trial.value)

# Initialize the model with the best parameters from the best trial
model = CatBoostClassifier(**best_trial.params, random_seed=42, verbose=False, eval_metric='AUC')

# Fit the model on the complete training dataset
model.fit(X, y, cat_features=categorical_features)

# Make predictions on the test dataset
final_test_predictions = model.predict(test)

# Create dataframe with test IDs and predictions
results_df = pd.DataFrame({
    'id': test_ids,
    'Depression': final_test_predictions
})

# Save results to CSV without index
filename = f'submission_catboost_{best_trial.value:.4f}.csv'
results_df.to_csv(filename, index=False)

print(f"Results saved to {filename}")
print("\nFirst few rows of results:")
print(results_df.head())

# LGBM

In [115]:
import warnings
from lightgbm import LGBMClassifier
import optuna
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import logging 

# Suppress warnings
# Suppress LightGBM warnings
logging.getLogger('lightgbm').setLevel(logging.ERROR)

# Suppress all warnings
warnings.filterwarnings("ignore")

categorical_features = X.select_dtypes(include=['object']).columns.tolist()

for feature in categorical_features:
    X[feature] = X[feature].astype('category')

# Define the objective function for LightGBM
def lgbm_objective(trial):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 2, 16),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.25),
        'n_estimators': trial.suggest_int('n_estimators', 150, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 250),
        'subsample': trial.suggest_float('subsample', 0.1, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.02, 0.9),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.05, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.05, 2.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 2),
        'random_state': 42,
        'metric': 'accuracy',
        'objective': 'binary',
        'n_jobs': 3  # Enable the use of all available threads
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    for train_index, test_index in skf.split(X, y):
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        model = LGBMClassifier(**param, verbosity = 0)  # Removed verbosity from here
        model.fit(X_train_fold, y_train_fold)
        preds = model.predict(X_test_fold)
        accuracy = accuracy_score(y_test_fold, preds)
        accuracies.append(accuracy)
    return np.mean(accuracies)

# Create a study object for LightGBM
lgbm_study = optuna.create_study(direction="maximize")
lgbm_study.optimize(lgbm_objective, n_trials=15, n_jobs=3)

# Train the best model found and test it on X
lgbm_best_params = lgbm_study.best_trial.params
lgbm_best_model = LGBMClassifier(**lgbm_best_params, random_state=42, n_jobs=3, metric='accuracy', verbosity=0)  # Removed verbosity from here

# Print best trial and best model results


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    lgbm_best_model.fit(X_train_fold, y_train_fold)
    lgbm_preds_fold = lgbm_best_model.predict(X_test_fold)
    fold_accuracy = accuracy_score(y_test_fold, lgbm_preds_fold)
    accuracies.append(fold_accuracy)

mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy across all folds: {mean_accuracy:.4f}")
print("Best Trial Value:", lgbm_study.best_trial.value)

# Print classification report for the best performing LGBM model
print("\nClassification Report for Best Performing LGBM Model:")
print(classification_report(y, lgbm_best_model.predict(X)))


[I 2024-11-13 20:39:26,994] A new study created in memory with name: no-name-3839175a-8f72-4d26-bb6c-857edf28ac3c


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[I 2024-11-13 20:39:36,140] Trial 1 finished with value: 0.9363752665245203 and parameters: {'num_leaves': 46, 'max_depth': 7, 'learning_rate': 0.18534012813331913, 'n_estimators': 332, 'min_child_samples': 142, 'subsample': 0.8592575944071619, 'colsample_bytree': 0.43364580519941487, 'reg_alpha': 1.6528961011626473, 'reg_lambda': 0.15577571670160345, 'scale_pos_weight': 1.4077649585105103}. Best is trial 1 with value: 0.9363752665245203.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-11-13 20:39:47,079] Trial 2 finished with value: 0.9335963041933191 and parameters: {'num_leaves': 53, 'max_depth': 6, 'learning_rate': 0.20148986546599099, 'n_estimators': 780, 'min_child_samples': 26, 'subsample': 0.42624332801906734, 'colsample_bytree': 0.6257522382315681, 'reg_alpha': 0.4197388989765541, 'reg_lambda': 1.3938776157888881, 'scale_pos_weight': 0.7072459112477489}. Best is trial 1 with value: 0.9363752665245203.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-11-13 20:40:39,250] Trial 14 finished with value: 0.9375266524520256 and parameters: {'num_leaves': 5, 'max_depth': 16, 'learning_rate': 0.018088590866226703, 'n_estimators': 714, 'min_child_samples': 245, 'subsample': 0.6550470332013069, 'colsample_bytree': 0.3904687168048067, 'reg_alpha': 1.1141748982411714, 'reg_lambda': 1.7890617589608413, 'scale_pos_weight': 0.9529124599839762}. Best is trial 13 with value: 0.9392466240227435.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[I 2024-11-13 20:40:43,019] Trial 11 finished with value: 0.935188343994314 and parameters: {'num_leaves': 98, 'max_depth': 16, 'learning_rate': 0.09148390917314152, 'n_estimators': 858, 'min_child_samples': 159, 'subsample': 0.2680939699234666, 'colsample_bytree': 0.5748183933445424, 'reg_alpha': 0.25967047092628665, 'reg_lambda': 1.1698201520052633, 'scale_pos_weight': 1.323906386884452}. Best is trial 13 with value: 0.9392466240227435.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Mean Accuracy across all folds: 0.9392
Best Trial Value: 0.9392466240227435

Classification Report for Best Performing LGBM Model:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96    115133
           1       0.86      0.80      0.83     25567

    accuracy                           0.94    140700
   macro avg       0.91      0.89      0.90    140700
weighted avg       0.94      0.94      0.94    140700



In [39]:
# Retrieve the best model from the study
best_model_trial = lgbm_study.best_trial
best_model_params = best_model_trial.params

# Train the best model
best_model = LGBMClassifier(**best_model_params, random_state=42, silent=True, n_jobs=3)
best_model.fit(X, y)

# Predict on the training data
train_predictions = best_model.predict(X)
train_accuracy = accuracy_score(y, train_predictions)
print(f"Best Model Training Accuracy: {train_accuracy:.4f}")

# Make predictions on the test dataset
final_test_predictions = best_model.predict(test)

# Create dataframe with test IDs and predictions
results_df = pd.DataFrame({
    'id': test_ids,
    'Depression': final_test_predictions
})

# Save results to CSV with accuracy in the filename
results_df.to_csv(f'submission_lgbm_{train_accuracy:.4f}.csv', index=False)

print(f"Results saved to submission_lgbm_{train_accuracy:.4f}.csv")
print("\nFirst few rows of results:")
print(results_df.head())


[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import joblib

# Set CatBoost to use 95 threads to avoid TBB warnings
MAX_THREADS = 95

# Define categorical features
categorical_features = ['Gender', 'City', 'Working Professional or Student', 'Profession', 'Degree']

# Objective function for Optuna optimization
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 30, 300),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 20.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 300),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.000001, 20.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 20.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 2),
        'cat_features': categorical_features,
        'thread_count': MAX_THREADS,  # Set to 95 to avoid TBB warnings
        'random_seed': 42,
        'verbose': False,
        'early_stopping_rounds': 50,
        'loss_function': 'Logloss',
        'border_count': trial.suggest_int('border_count', 128, 254),
        'eval_metric': 'Accuracy'
    }

    # Cross-validation setup
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    def train_fold(train_index, test_index):
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        model = CatBoostClassifier(**param)
        model.fit(X_train_fold, y_train_fold, cat_features=categorical_features)
        preds = model.predict(X_test_fold)
        return accuracy_score(y_test_fold, preds)

    # Run folds in parallel with controlled `n_jobs`
    fold_accuracies = joblib.Parallel(n_jobs=5)(joblib.delayed(train_fold)(train_idx, test_idx) for train_idx, test_idx in skf.split(X, y))
    
    return np.mean(fold_accuracies)

# Optimize using a controlled number of parallel trials
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, n_jobs=5)  # Set n_jobs to avoid excessive parallelism

# Retrieve and train the best model on all folds
top_model = study.best_trial
model_params = top_model.params
best_model = CatBoostClassifier(**model_params, random_seed=42, verbose=False, early_stopping_rounds=50, eval_metric='Accuracy')

# Cross-validation for best model
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def train_fold(train_index, test_index):
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    model = CatBoostClassifier(**model_params)
    model.fit(X_train_fold, y_train_fold, cat_features=categorical_features)
    preds = model.predict(X_test_fold)
    return accuracy_score(y_test_fold, preds)

fold_accuracies = joblib.Parallel(n_jobs=5)(
    joblib.delayed(train_fold)(train_idx, test_idx) for train_idx, test_idx in skf.split(X, y)
)

mean_accuracy = np.mean(fold_accuracies)
print(f"Mean accuracy for the best model: {mean_accuracy:.4f}")

# Save parameters and accuracy to a file once after all trials
with open("catboost_model_performance.txt", "a") as file:
    file.write(f"Params: {top_model.params}, Validation Accuracy: {mean_accuracy:.4f}\n")


In [ ]:
# # Load the best model parameters from the file and create seven LightGBM models

# #0.9394
# best_params_0 = {
#     'num_leaves': 20,
#     'max_depth': 16,
#     'learning_rate': 0.0848416870074553,
#     'n_estimators': 206,
#     'min_child_samples': 107,
#     'subsample': 0.5711321624766442,
#     'colsample_bytree': 0.7798078541869621,
#     'reg_alpha': 1.2275161885313777,
#     'reg_lambda': 0.9975430987118526,
#     'scale_pos_weight': 0.9789268184079968,
#     'random_state': 42,
#     'metric': 'accuracy',
#     'objective': 'binary'
# }
# best_params_6 = {
#     'num_leaves': 6,
#     'max_depth': 10,
#     'learning_rate': 0.08463134383900794,
#     'n_estimators': 815,
#     'min_child_samples': 160,
#     'subsample': 0.25674611723848206,
#     'colsample_bytree': 0.292312742969389,
#     'reg_alpha': 0.062499135989303845,
#     'reg_lambda': 0.5895089625561618,
#     'scale_pos_weight': 1.0183230641517838,
#     'random_state': 42,
#     'metric': 'accuracy',
#     'objective': 'binary'
# }

# best_params_7 = {
#     'num_leaves': 5,
#     'max_depth': 16,
#     'learning_rate': 0.13496081786548303,
#     'n_estimators': 590,
#     'min_child_samples': 41,
#     'subsample': 0.7055733486544014,
#     'colsample_bytree': 0.3408694828026245,
#     'reg_alpha': 0.5876000905323011,
#     'scale_pos_weight': 1.14899698370991,
#     'reg_lambda': 0.9048763397060431,
#     'random_state': 42,
#     'metric': 'accuracy',
#     'objective': 'binary'
# }

# # best_params_1 = {
# #     'num_leaves': 10,
# #     'max_depth': 13,
# #     'learning_rate': 0.07816115162882467,
# #     'n_estimators': 228,
# #     'min_child_samples': 22,
# #     'subsample': 0.4938200083689593,
# #     'colsample_bytree': 0.4473341130299121,
# #     'reg_alpha': 1.503635888471512,
# #     'reg_lambda': 0.5820095417203568,
# #     'scale_pos_weight': 1.0287016668121143,
# #     'random_state': 42,
# #     'metric': 'accuracy',
# #     'objective': 'binary'
# # }

# # best_params_2 = {
# #     'num_leaves': 7,
# #     'max_depth': 7,
# #     'learning_rate': 0.08684584847555041,
# #     'n_estimators': 459,
# #     'min_child_samples': 83,
# #     'scale_pos_weight': 1.0287016668121143,
# #     'subsample': 0.5810616462037733,
# #     'colsample_bytree': 0.3749110968508681,
# #     'reg_alpha': 0.750822476761735,
# #     'reg_lambda': 0.16643264777962907,
# #     'random_state': 42,
# #     'metric': 'accuracy',
# #     'objective': 'binary'
# # }

# # best_params_3 = {
# #     'num_leaves': 21,
# #     'max_depth': 6,
# #     'learning_rate': 0.08870453392237598,
# #     'n_estimators': 171,
# #     'min_child_samples': 40,
# #     'subsample': 0.13794790119898281,
# #     'scale_pos_weight': 1.0287016668121143,
# #     'colsample_bytree': 0.33234701483637774,
# #     'reg_alpha': 0.35384112403630114,
# #     'reg_lambda': 0.21776834318291055,
# #     'random_state': 42,
# #     'metric': 'accuracy',
# #     'objective': 'binary'
# # }

# # best_params_4 = {
# #     'num_leaves': 12,
# #     'max_depth': 12,
# #     'learning_rate': 0.13552344141193437,
# #     'n_estimators': 312,
# #     'min_child_samples': 115,
# #     'subsample': 0.5061868744743441,
# #     'colsample_bytree': 0.15087353298868322,
# #     'reg_alpha': 1.421372562047065,
# #     'reg_lambda': 0.7840468800301836,
# #     'random_state': 42,
# #     'scale_pos_weight': 1.0287016668121143,
# #     'metric': 'accuracy',
# #     'objective': 'binary'
# # }

# # best_params_5 = {
# #     'num_leaves': 28,
# #     'max_depth': 16,
# #     'learning_rate': 0.010556540477690654,
# #     'n_estimators': 995,
# #     'min_child_samples': 114,
# #     'subsample': 0.8502140212699687,
# #     'colsample_bytree': 0.3070518212603518,
# #     'reg_alpha': 1.2316643109343381,
# #     'reg_lambda': 1.2336271766305382,
# #     'random_state': 42,
# #     'scale_pos_weight': 0.9066902282314742,
# #     'metric': 'accuracy',
# #     'objective': 'binary'
# # }







# # Create and train seven LightGBM models with the best parameters
# models = [LGBMClassifier(**params) for params in [best_params_0, best_params_6, best_params_7]]
# for model in models:
#     model.fit(X_train, y_train)

# # Predict on X_test using the trained models and ensemble their outputs
# ensemble_predictions = [model.predict(X_test) for model in models]

# # Convert predictions to numpy array and get majority vote
# ensemble_predictions = np.array(ensemble_predictions)
# final_ensemble_predictions = np.apply_along_axis(lambda x: np.bincount(x, minlength=2).argmax(), axis=0, arr=ensemble_predictions)

# # Calculate ensemble accuracy on X_test
# ensemble_accuracy = accuracy_score(y_test, final_ensemble_predictions)
# print(f"Ensemble Model Accuracy on X_test: {ensemble_accuracy:.4f}")

# # Predict on test dataset using the trained models and ensemble their outputs
# test_predictions = [model.predict(test) for model in models]

# # Convert predictions to numpy array and get majority vote
# test_predictions = np.array(test_predictions)
# final_test_predictions = np.apply_along_axis(lambda x: np.bincount(x, minlength=2).argmax(), axis=0, arr=test_predictions)

# # Create dataframe with test IDs and predictions
# results_df = pd.DataFrame({
#     'id': test_ids,
#     'Depression': final_test_predictions
# })

# # Save results to CSV without index
# results_df.to_csv(f'submission_lgbm_{ensemble_accuracy:.4f}.csv', index=False)

# print("Results saved to submission_lgbm.csv")
# print("\nFirst few rows of results:")
# print(results_df.head())